In [1]:
import pandas as pd

# SnpSift Concordance Output 
### Comparing imputed hu1AF4ED_23andMe_imputed.vcf and hu1AF4ED_CG_imputed.vcf files

In [ ]:
path = path = '/Users/jerenolsen/Desktop/All_Tests/bcftools_and_concordance/testing_concordance/snpSift_concordance_ouput/concordance_hu1AF4ED_23andMe_imputed_hu1AF4ED_CGI_imputed.by_sample.txt'

In [133]:
def read_concordance(conc_file):
    cols = None
    with open(conc_file, 'r') as f:
        for line in f:
            cols = line
            break
            
    cols = line.strip('\n')
    cols = cols.rsplit("\t")
    cols.remove('')
    cols.remove('')
    cols.remove('')
    cols.insert(1, 'Unknown3')
    cols.insert(1, 'Unknown2')
    cols.insert(1, 'Unknown1')
    
    cols.append('Unknown')
    col_types = {col:str for col in cols}
    
    df = pd.read_csv(conc_file, skiprows = 1, sep='\t', names = cols, header = None, dtype=col_types)
    return df

In [135]:
df = read_concordance(path)

In [143]:
def get_changed_counts(df):
    # Columns with '/' indicate sameness or difference between the two allele types the '/' separates
    # ie: ALT_1/REF -> ALT_1 position in file 1 changed to REF in file 2
    conc_sample = df
    same = 0
    total = 0
    for col in conc_sample.keys():
        items = col.split('/')
        if len(items) <= 1: #Skip row because no '/'
            continue
        elif items[0] == items[1]: #Example: ALT_1/ALT_1
            same+=int(conc_sample[col])

        try:
            total += int(conc_sample[col]) #Example: ALT_1/REF
        except Exception as e:
            continue
            
    return same, total

In [144]:
same, total = get_changed_counts(df)

In [145]:
alt1_to_ref = int(conc_sample['ALT_1/REF'])
alt1_to_alt2 = int(conc_sample['ALT_1/ALT_2'])

alt2_to_alt1 = int(conc_sample['ALT_2/ALT_1'])
alt2_to_ref = int(conc_sample['ALT_2/REF'])

ref_to_alt1 = int(conc_sample['REF/ALT_1'])
ref_to_alt2 = int(conc_sample['REF/ALT_2'])

In [193]:
print(f"Total Positions: {total}")
print(f"Changed Positions: {total-same}")
print(f"Changed Percent: {round(100 - same/total*100,2)}%")

Total Positions: 30761499
Changed Positions: 445761
Changed Percent: 1.45%


In [147]:
print(f"Changed Heterozygous ALT to Homozygous REF : {alt1_to_ref}, {round((alt1_to_ref/total)*100,2)}% ")
print(f"Changed Heterozygous ALT to Homozygous ALT: {alt1_to_alt2}, {round((alt1_to_alt2/total)*100,2)}% ")

Changed Heterozygous ALT to Homozygous REF : 189788, 0.62% 
Changed Heterozygous ALT to Homozygous ALT: 49242, 0.16% 


In [148]:
print(f"Changed Homozygous ALT to Heterozygous REF : {alt2_to_alt1}, {round((alt2_to_alt1/total)*100,2)}% ")
print(f"Changed Homozygous ALT to Homozygous REF: {alt2_to_ref}, {round((alt2_to_ref/total)*100,2)}% ")

Changed Homozygous ALT to Heterozygous REF : 44843, 0.15% 
Changed Homozygous ALT to Homozygous REF: 9114, 0.03% 


In [149]:
print(f"Changed Homozygous REF to Heterozygous ALT : {ref_to_alt1}, {round((ref_to_alt1/total)*100,2)}% ")
print(f"Changed Homozygous REF to Homozygous ALT: {ref_to_alt2}, {round((ref_to_alt2/total)*100,2)}% ")

Changed Homozygous REF to Heterozygous ALT : 146393, 0.48% 
Changed Homozygous REF to Homozygous ALT: 6293, 0.02% 


In [182]:
from tabulate import tabulate

data = [
    ["Heterozygous ALT to Homozygous REF", alt1_to_ref, f"{round((alt1_to_ref/total)*100,2)}%"],
    ["Heterozygous ALT to Homozygous ALT", alt1_to_alt2, f"{round((alt1_to_alt2/total)*100,2)}%"],
    ["Homozygous ALT to Heterozygous REF", alt2_to_alt1, f"{round((alt2_to_alt1/total)*100,2)}%"],
    ["Homozygous ALT to Homozygous REF", alt2_to_ref, f"{round((alt2_to_ref/total)*100,2)}%"],
    ["Homozygous REF to Heterozygous ALT", ref_to_alt1, f"{round((ref_to_alt1/total)*100,2)}%"],
    ["Homozygous REF to Homozygous ALT", ref_to_alt2, f"{round((ref_to_alt2/total)*100,2)}%"]
]

In [183]:
print(tabulate(data, headers=["Allele Change Type", "Count", "Percentage"], tablefmt="fancy_grid", numalign="right"))

╒════════════════════════════════════╤═════════╤══════════════╕
│ Allele Change Type                 │   Count │ Percentage   │
╞════════════════════════════════════╪═════════╪══════════════╡
│ Heterozygous ALT to Homozygous REF │  189788 │ 0.62%        │
├────────────────────────────────────┼─────────┼──────────────┤
│ Heterozygous ALT to Homozygous ALT │   49242 │ 0.16%        │
├────────────────────────────────────┼─────────┼──────────────┤
│ Homozygous ALT to Heterozygous REF │   44843 │ 0.15%        │
├────────────────────────────────────┼─────────┼──────────────┤
│ Homozygous ALT to Homozygous REF   │    9114 │ 0.03%        │
├────────────────────────────────────┼─────────┼──────────────┤
│ Homozygous REF to Heterozygous ALT │  146393 │ 0.48%        │
├────────────────────────────────────┼─────────┼──────────────┤
│ Homozygous REF to Homozygous ALT   │    6293 │ 0.02%        │
╘════════════════════════════════════╧═════════╧══════════════╛


In [191]:
data = [
    ['Total Positions Count', total],
    ["Changed Positions Count", total-same],
    ["Changed Percent", round(100 - same/total*100,2)]
    
]

In [192]:
print(tabulate(data, headers=["Position Similarities",'Measure'], tablefmt="fancy_grid", numalign="right"))

╒═════════════════════════╤═════════════╕
│ Position Similarities   │     Measure │
╞═════════════════════════╪═════════════╡
│ Total Positions Count   │ 3.07615e+07 │
├─────────────────────────┼─────────────┤
│ Changed Positions Count │      445761 │
├─────────────────────────┼─────────────┤
│ Changed Percent         │        1.45 │
╘═════════════════════════╧═════════════╛
